In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import random
from empath import Empath
from sklearn.preprocessing import StandardScaler

def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)

set_seed(1)
lexicon = Empath()

def prepare_data(dataset_path_train = None,):
    if (dataset_path_train is None):
        raise RuntimeException("Error! Dataset must be provided")
    train = pd.read_csv(dataset_path_train)
    y_train = train['label']
    X_train = train[['roberta_prediction', 'text']]
    return X_train, y_train

def empathfeats(X, train=False, sclr=None):
    rows = []
    for t in X['text']:
        empath = lexicon.analyze(t, normalize=True)
        rows.append(pd.DataFrame([empath]))
    df = pd.concat(rows, ignore_index=True)
    X = pd.concat([X,df], axis=1)
    
    X = X.drop(columns=['text'])
    to_drop = ['health','banking','night','college','exasperation','reading','worship','eating','water','legend','neglect','swimming','love','sympathy','vehicle','disgust','sound','sailing','warmth','fun','joy','affection','lust','shame','anger','car','technology','power','white_collar_job','party','cleaning','competing','ocean','contentment','musical']
    X = X.drop(columns=to_drop)
    
    if(train): 
        sclr = StandardScaler()
        X = sclr.fit_transform(X)
    else:
        X = sclr.transform(X)
    return X, sclr

def robertafeat(text):
    df = pd.DataFrame({'post_id': [0],'text': [text]})
    prediction = 1
    return prediction
    
def train_model():
    X_train, y_train = prepare_data(dataset_path_train = 'train_pred_mental_health.csv')
    X_train, sclr = empathfeats(X_train, train=True)
    hp = {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
    model = LogisticRegression(max_iter=1000000, **hp)
    model.fit(X_train, y_train)
    return model, sclr

def make_prediction(text, model, sclr):
    trans, sclr = empathfeats(pd.DataFrame({'roberta_prediction': [robertafeat(text)], 'text': [text]}), sclr=sclr)
    return model.predict(trans)

def final_prediction(text):
    global model, sclr
    return make_prediction(text, model, sclr)[0]

model, sclr = train_model()

# Demo :)

In [ ]:
while(True):
    txt = input()
    pred = final_prediction(txt)
    if pred == 0:
        print("Not stressful")
    else:
        print("Stressful")